In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_text_label.csv


In [4]:
!pip install keras
!pip install matplotlib-venn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
df_amzn = pd.read_csv('Amzn_NewsPriceHistDirection.csv')
df_aapl = pd.read_csv('Aapl_NewsPriceHistDirection.csv')
df = [df_amzn, df_aapl]
df = pd.concat(df, ignore_index=True)
df

,uuid,author,url,title,language,text,published,title_aapl,text_aapl,title_amzn,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
0,3b7327f6858be72e780b02b863d941fe7ce81eab,NaN,https://money.cnn.com/2018/09/05/investing/pre...,6 things to know before the opening bell,english,Click chart for more in-depth data. 1. Big Tec...,2018-09-05 05:12:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
1,8a3015a274df99d231fcf7ff3eb38e2660a984ff,Michael Wiggins De Oliveira,https://seekingalpha.com/article/4203857-netfl...,"Netflix And The Competition - Netflix, Inc. (N...",english,Investment Thesis Netflix ( NFLX ) is a remark...,2018-09-04 15:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
2,df5883d078ebfbe015e1e717002e4b889647dbee,Larry Ramer,https://investorplace.com/2018/09/pandora-stoc...,Pandora Stock Gets a Boost — Buy Now,english,MannKind Surges More Than 50% >>> READ MORE\nB...,2018-09-04 16:22:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
3,861c19bff56d09f97b6798f5fbe374061e8aa55b,morningstar.com,http://news.morningstar.com/all/dow-jones/us-m...,MARKET SNAPSHOT: Tech Tumble Weighs On Stocks;...,english,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
4,cb960bebc2bf78d327026127c9fe919a4159e80d,morningstar.com,http://news.morningstar.com/all/benzinga/BenzB...,"New Month, Same Old Story: Trade Fears Persist...",english,"It may be a new month, but the same trade conc...",2018-09-05 00:00:00+00:00,False,True,False,...,1896.99,1885.99,1896.00,1882.98,1861.48,1847.11,1822.94,1833.98,1796.14,1787.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88696,c863acca384e58ae60edade344cebc64ba665be2,Motley Fool,http://www.nasdaq.com/article/amazon-owns-the-...,"Amazon Owns the Smart-Speaker Market, but Goog...",english,Amazon.com (NASDAQ: AMZN) continues to dominat...,2018-06-07 11:19:00+00:00,False,True,True,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
88697,a8a1de01d93898c18d9b2626a4d8cbfa32e883aa,Jussi Askola,https://seekingalpha.com/article/4179901-landl...,A Landlord's Dream - High Return With Lower Ri...,english,Source\nInvesting capital is all about balanci...,2018-06-07 13:00:00+00:00,False,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
88698,091d917711b14f9693b988e17ebdf950e3a3a012,Marguerite Chambers,https://heraldks.com/partnervest-advisory-serv...,Partnervest Advisory Services Position in Appl...,english,Partnervest Advisory Services Position in Appl...,2018-06-06 20:13:00+00:00,True,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
88699,fa0f6039826857a1a0a0520821c1ce399b7b6d23,Darrin Black,https://www.whatsonthorold.com/2018/06/06/enco...,Encore Capital Group In (ECPG) Holder Public E...,english,Encore Capital Group In (ECPG) Holder Public ...,2018-06-06 20:49:00+00:00,False,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36


In [23]:
df = df[["title", "text", "direction"]]
df = df.replace(-1, 0)

In [24]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

In [25]:
title=df['title'].tolist()
label=df['direction'].tolist()

print(title[0])
print(label[0])

6 things to know before the opening bell
0


In [26]:
title_tokenized, title_tokenizer=tokenize(title)
title_padded=pad_sequences(title_tokenized, padding='post')

In [27]:
class_label = pd.get_dummies(label).values

In [28]:
print(title_padded[0])

[157 187   3 183 299   2 883 709   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0]


In [29]:
from sklearn.model_selection import train_test_split
# trainY = pd.get_dummies(label.values).values
# print(trainY.shape)
#X_train, X_test, Y_train, Y_test = train_test_split(title_padded_amzn, class_label_amzn, test_size=0.2, random_state=56)
X_train, X_test, Y_train, Y_test = train_test_split(title_padded, class_label, test_size=0.2, random_state=56)

In [30]:
TrainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
TestX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
TrainX = TrainX.astype('float64')
TestX = TestX.astype('float64')

In [31]:
TrainX.shape

(70960, 1, 60)

In [32]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense

In [34]:
from keras.optimizers import Adam
layers = [LSTM(128,input_shape=(1,TrainX.shape[2]),dropout=0.2), Dense(2,activation='sigmoid')]
model = tf.keras.models.Sequential(layers)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),metrics=['accuracy'])
model.fit(TrainX, Y_train, epochs=100, batch_size=64, verbose=2,validation_split=0.2)

Epoch 1/100
887/887 - 6s - loss: 0.7238 - accuracy: 0.5094 - val_loss: 0.7159 - val_accuracy: 0.5116 - 6s/epoch - 7ms/step
Epoch 2/100
887/887 - 3s - loss: 0.7130 - accuracy: 0.5110 - val_loss: 0.7122 - val_accuracy: 0.5101 - 3s/epoch - 4ms/step
Epoch 3/100
887/887 - 3s - loss: 0.7099 - accuracy: 0.5098 - val_loss: 0.7080 - val_accuracy: 0.5176 - 3s/epoch - 4ms/step
Epoch 4/100
887/887 - 3s - loss: 0.7054 - accuracy: 0.5116 - val_loss: 0.7074 - val_accuracy: 0.5107 - 3s/epoch - 4ms/step
Epoch 5/100
887/887 - 3s - loss: 0.7060 - accuracy: 0.5099 - val_loss: 0.7071 - val_accuracy: 0.5058 - 3s/epoch - 4ms/step
Epoch 6/100
887/887 - 3s - loss: 0.7036 - accuracy: 0.5122 - val_loss: 0.7056 - val_accuracy: 0.5098 - 3s/epoch - 3ms/step
Epoch 7/100
887/887 - 4s - loss: 0.7026 - accuracy: 0.5122 - val_loss: 0.7056 - val_accuracy: 0.5069 - 4s/epoch - 4ms/step
Epoch 8/100
887/887 - 3s - loss: 0.7018 - accuracy: 0.5119 - val_loss: 0.7032 - val_accuracy: 0.5163 - 3s/epoch - 4ms/step
Epoch 9/100
887/

In [35]:
model.evaluate(TestX, Y_test, verbose=2)

555/555 - 1s - loss: 0.6899 - accuracy: 0.5302 - 790ms/epoch - 1ms/step


[0.6898715496063232, 0.5301843285560608]